In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# **Importing Data and search the low cardinality columns and the numerical ones**

In [13]:

from sklearn.model_selection import train_test_split

X=pd.read_csv('../input/titanic/train.csv')
X_test_full=pd.read_csv('../input/titanic/test.csv')

X.dropna(axis=0, subset=['Survived'], inplace=True)
y=Titanic_train.Survived
X.drop(['Survived'], axis=1, inplace=True)

# Splitting the training data into training and validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=1)
#Searching for the categorical columns with unique values < 10
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]
# Searching for the numerical columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]


cols=low_cardinality_cols+numeric_cols
X_train = X_train_full[cols].copy()
X_valid = X_valid_full[cols].copy()
X_test = X_test_full[cols].copy()





# **Searching for safe to ordinally encode categorical columns**

In [14]:
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# **Creating our preprocessor with Simple Imputing and Categorical transformer**

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder


numerical_transformer = SimpleImputer(strategy='mean')


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('Ordi', OrdinalEncoder(handle_unknown='error'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_cols),
        ('cat', categorical_transformer, low_cardinality_cols)
    ])



# **Creating our model**

In [36]:
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error


model = XGBClassifier(random_state=0,n_estimators=500,learning_rate=0.05)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf.fit(label_X_train, y_train)

preds = clf.predict(label_X_valid)
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 0.22905027932960895


In [42]:
my_predictions=clf.predict(X_test)

,Sex,Embarked,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,male,Q,892,3,34.5,0,0,7.8292
1,female,S,893,3,47.0,1,0,7.0000
2,male,Q,894,2,62.0,0,0,9.6875
3,male,S,895,3,27.0,0,0,8.6625
4,female,S,896,3,22.0,1,1,12.2875
...,...,...,...,...,...,...,...,...
413,male,S,1305,3,NaN,0,0,8.0500
414,female,C,1306,1,39.0,0,0,108.9000
415,male,S,1307,3,38.5,0,0,7.2500
416,male,S,1308,3,NaN,0,0,8.0500


In [46]:
output = pd.DataFrame({'PassengerId': X_test.PassengerId,
                       'Survived': my_predictions})
output.to_csv('submission.csv', index=False)